# AOG Embed Service Example

This notebook demonstrates how to use AOG Embed API to generate text embedding vectors.

## Table of Contents
1. [Environment Setup](#1-Environment Setup)
2. [Helper Functions](#2-Helper Functions)
3. [Basic Embedding Generation](#3-Basic Embedding Generation)
4. [Similarity Calculation](#4-Similarity Calculation)
5. [Semantic Search](#5-Semantic Search)

## 1. Environment Setup

In [ ]:
import requests
import json
import numpy as np
from typing import List, Optional, Dict

# AOG service configuration
AOG_BASE_URL = "http://localhost:16688"
AOG_API_SPEC_VERSION = "v0.2"
DEFAULT_MODEL = "text-embedding-v3"

print("✅ Configuration complete")
print(f"📍 AOG service address: {AOG_BASE_URL}")
print(f"🤖 Default model: {DEFAULT_MODEL}")

## 2. Helper Functions

In [ ]:
def call_aog_embed(
    texts: List[str],
    model: str = DEFAULT_MODEL
) -> Optional[Dict]:
    """
    Call AOG Embed API
    
    Args:
        texts: List of texts
        model: Model name
    
    Returns:
        API response data
    """
    try:
        url = f"{AOG_BASE_URL}/aog/{AOG_API_SPEC_VERSION}/services/embed"
        
        payload = {
            "model": model,
            "input": texts
        }
        
        print(f"📤 Sending request...")
        print(f"📝 文本数量: {len(texts)}")
        
        response = requests.post(url, json=payload, timeout=60)
        response.raise_for_status()
        
        print("✅ 嵌入Generation successful")
        return response.json()
        
    except requests.exceptions.ConnectionError:
        print("❌ 无法连接到 AOG 服务")
        return None
    except Exception as e:
        print(f"❌ 错误: {e}")
        return None

def get_embeddings(texts: List[str], model: str = DEFAULT_MODEL) -> Optional[np.ndarray]:
    """
    获取文本嵌入向量
    
    Args:
        texts: List of texts
        model: Model name
    
    Returns:
        嵌入向量数组 (numpy array)
    """
    response = call_aog_embed(texts, model)
    
    if not response or "data" not in response:
        return None
    
    # 提取嵌入向量
    embeddings = []
    for item in response["data"]:
        embeddings.append(item["embedding"])
    
    return np.array(embeddings)

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """
    计算两个向量余弦相似度
    
    Args:
        vec1: 向量1
        vec2: 向量2
    
    Returns:
        相似度分数 (0-1)
    """
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

def find_most_similar(query_embedding: np.ndarray, doc_embeddings: np.ndarray, docs: List[str], top_k: int = 3):
    """
    找到最相似文档
    
    Args:
        query_embedding: 查询嵌入向量
        doc_embeddings: 文档嵌入向量数组
        docs: 文档List of texts
        top_k: Returns前 k 个最相似文档
    
    Returns:
        最相似文档及其相似度
    """
    similarities = []
    
    for i, doc_emb in enumerate(doc_embeddings):
        sim = cosine_similarity(query_embedding, doc_emb)
        similarities.append((i, sim, docs[i]))
    
    # 按相似度排序
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return similarities[:top_k]

print("✅ Helper Functions定义完成")

## 3. Basic Embedding Generation

In [ ]:
# 基本示例
texts = [
    "人工智能正在改变世界",
    "机器学习是AI一个Important分支",
    "深度学习使用神经网络"
]

response = call_aog_embed(texts)

if response:
    print("\n📊 响应信息:")
    print(f"模型: {response.get('model')}")
    print(f"请求ID: {response.get('id')}")
    print(f"嵌入数量: {len(response.get('data', []))}")
    
    # 显示第一个嵌入向量维度
    if response.get('data'):
        first_embedding = response['data'][0]['embedding']
        print(f"向量维度: {len(first_embedding)}")
        print(f"向量前10个值: {first_embedding[:10]}")

## 4. Similarity Calculation

In [ ]:
# 计算文本相似度
text_pairs = [
    ("我喜欢吃苹果", "我喜欢吃水果"),
    ("我喜欢吃苹果", "今天天气很好"),
    ("机器学习很有趣", "深度学习很强大")
]

print("\n📊 文本Similarity Calculation:")
print("="*60)

for text1, text2 in text_pairs:
    # 获取嵌入
    embeddings = get_embeddings([text1, text2])
    
    if embeddings is not None and len(embeddings) == 2:
        # 计算相似度
        similarity = cosine_similarity(embeddings[0], embeddings[1])
        
        print(f"\n文本1: {text1}")
        print(f"文本2: {text2}")
        print(f"相似度: {similarity:.4f}")
        print("-" * 60)

print("="*60)

## 5. Semantic Search

演示如何使用嵌入向量forSemantic Search

In [ ]:
# 文档库
documents = [
    "Python是一种流行编程语言",
    "机器学习可以用于数据分析",
    "深度学习是机器学习子集",
    "自然语言处理处理文本数据",
    "计算机视觉处理图像数据",
    "强化学习通过试错来学习",
    "神经网络模拟人脑结构",
    "数据科学结合统计和编程"
]

# 生成文档嵌入
print("📚 生成文档嵌入...")
doc_embeddings = get_embeddings(documents)

if doc_embeddings is not None:
    print(f"✅ 已生成 {len(doc_embeddings)} 个文档嵌入向量\n")
    
    # 查询示例
    queries = [
        "什么是深度学习?",
        "如何处理文本?",
        "编程语言有哪些?"
    ]
    
    for query in queries:
        print("\n" + "="*60)
        print(f"🔍 查询: {query}")
        print("="*60)
        
        # 生成查询嵌入
        query_embedding = get_embeddings([query])
        
        if query_embedding is not None:
            # 找到最相似文档
            results = find_most_similar(query_embedding[0], doc_embeddings, documents, top_k=3)
            
            print("\n📄 最相关文档:")
            for rank, (idx, similarity, doc) in enumerate(results, 1):
                print(f"\n{rank}. 相似度: {similarity:.4f}")
                print(f"   文档: {doc}")
    
    print("\n" + "="*60)

## 🎉 Summary

Through this notebook, you have learned:

1. ✅ 调用 Embed API 生成嵌入向量
2. ✅ 计算文本相似度
3. ✅ 实现Semantic Search
4. ✅ 处理批量文本嵌入

### 应用场景

- 🔍 **Semantic Search**: 根据语义而非关键词匹配查找相关内容
- 📊 **文本聚类**: 将相似文本分组
- 🎯 **推荐系统**: 基于内容相似度推荐
- 📝 **文本分类**: 使用嵌入向量作为特征
- 🔗 **去重**: 识别重复或相似文本

### Next Steps

- 尝试不同文本和查询
- 集成向量数据库(Milvus、Pinecone)
- 构建完整Semantic Search系统
- Check out [Chat 服务示例](../text-generation/)
- Read [AOG API 文档](../../docs/)